In [2]:
# Package utilisé
# pytorch : https://pytorch.org/
# numpy : https://numpy.org/
# matplotlib : https://matplotlib.org/
# pandas : https://pandas.pydata.org/
# scikit-learn : https://scikit-learn.org/stable/

In [4]:
from importlib.metadata import version
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import numpy as np
import math
from dataclasses import dataclass
from matplotlib import pyplot as plt
import time
import os

# Pour torch si vous avez un GPU
device = "cpu" if not torch.cuda.is_available() else "cuda"
# device = "cpu" # Pour forcer l'utilisation du CPU


In [ ]:
# Environement potentielement tester
from environnement.environnement import Environnement as env # mother class
from environnement.environnement1 import Environnement1 as env1
from environnement.environnement2Str import Environnement2 as env2Str
from environnement.environnement3Str import Environnement3 as env3Str
from environnement.environnement6Str import Environnement6 as env6Str
from environnement.small_loop import small_loop

# L'agent qui prédit l'avenir
Dans ce notebook nous allons implémenter un agent dévellopementale qui aura comme simple but de prédire le prochain feedback de son environement.

## Mécanisme de prédiction
Dans se notebook nous allons implémenté deux mécanismes de machine learning, un decision tree et un deep neural network. L'entrainement des deux modèls se fera sur la "vie" de l'agent. C'est à dire que les modèls s'entrainerons seulement sur les interactions que l'agents aura faites.

# Définition de l'agent

In [ ]:
class Agent1:
    def __init__(self, model):
        """ 
        Création de l'agent.
        
        - self._action : action précédente
        - self._predicted_outcome : prédiction de l'outcome précédent
        """
        self._action = None
        self._predicted_outcome = None
        self._model = model

    def fit(self, action, outcome):
        """
        Fonction d'entrainement de l'agent
        """
        try:
            self._model.train()
        except: # Si le model n'a pas de fonction 
            pass
        self._model.fit([action], [outcome])

    def predict(self, action):
        """
        Funciton de prédiction
        """
        try:
            self._model.eval()
        except: # Si le model n'a pas de fonction eval
            pass
        self._model.predict(action)

    def action(self, _outcome, env:env):
        """ 
        Fonction qui choisit l'action a faire en fonction de la dernière \
        intéraction avec l'environnement. \n
        C'est ici que nous allons implémenter un mécanisme de ML \
        pour choisir la prochaine action.

        :param: **_outcome** feedback de la dernière intéraction avec l'environnement

        :return: **action** action à effectuer
        """
        if self._action is not None:
            print(f"Action: {self._action}, Prediction: {self._predicted_outcome}, Outcome: {_outcome}, " 
                  f"Satisfaction: {self._predicted_outcome == _outcome}")
            # Nous entrainons le model sur la dernière action et son outcome
            self.fit(self._action, _outcome)
            # Nous faisons une prédiction sur une action aléatoire
            self._action = np.random.choice(env.get_actions())
            self._predicted_outcome = self.predict(self._action)
        else:
            # Au début de la vie de l'agent, nous prennons la première action possible
            self._action = env.get_actions()[0]
            print(f"Action de base : {self._action} aucune prédiciton pour la première action")
        
        return self._action

# Deep neural network